# SAT NETs relevant to the 2025 "Pivot to Chile", Phase 2

January 2025

This notebook calculates NETs for the "Phase 2" baseline SATs, and some variations on that.
The baseline model has:
1) a HWP, large enough to enable f/1.45 optics,
2) horn sizes about the same as SO, which coupled with the same f/# as SO, means that we roughly match the spillover and edge
taper of SO.

This modeling uses flat bands, and a single number for each band's spillover.

NETs are calculated per detector, per wafer, and "per tube", where horn-horn correlations, yield, the number of wafers/tube and the optically good fraction of detectors in a tube are all taken into account.

In [24]:
%pwd

'D:\\pgrimes\\Software\\GitHub\\bolo_calc_runs\\jbolo\\ChOpt_Report'

In [25]:

import sys
sys.path.insert(0,'D:\\pgrimes\\Software\\GitHub\\jbolo\\src')

os.environ['JBOLO_PATH'] = 'D:\\pgrimes\\Software\\GitHub\\jbolo'

In [26]:
print(sys.path)

['D:\\pgrimes\\Software\\GitHub\\jbolo\\src', 'D:\\pgrimes\\Software\\GitHub\\jbolo\\src', 'D:\\pgrimes\\Software\\GitHub\\jbolo\\src', 'D:\\pgrimes\\Software\\GitHub\\jbolo\\src', 'D:\\pgrimes\\Software\\GitHub\\jbolo\\src', 'D:\\Software\\miniconda3\\envs\\jbolo\\python313.zip', 'D:\\Software\\miniconda3\\envs\\jbolo\\DLLs', 'D:\\Software\\miniconda3\\envs\\jbolo\\Lib', 'D:\\Software\\miniconda3\\envs\\jbolo', '', 'D:\\Software\\miniconda3\\envs\\jbolo\\Lib\\site-packages', 'D:\\Software\\miniconda3\\envs\\jbolo\\Lib\\site-packages\\win32', 'D:\\Software\\miniconda3\\envs\\jbolo\\Lib\\site-packages\\win32\\lib', 'D:\\Software\\miniconda3\\envs\\jbolo\\Lib\\site-packages\\Pythonwin']


In [27]:
import os

import numpy as np
import matplotlib.pyplot as plt
import yaml
from jbolo_funcs import *

%matplotlib inline
plt.rcParams.update({'font.size': 12})
plt.rcParams['figure.figsize'] = [8, 4]

In [28]:
%cd "D:\\pgrimes\\Software\\GitHub\\bolo_calc_runs\\jbolo\\ChOpt_Report"
# List what yaml files you have so you can choose one for the next line.
%ls -lt1 yamls 

D:\pgrimes\Software\GitHub\bolo_calc_runs\jbolo\ChOpt_Report
 Volume in drive D is DATA
 Volume Serial Number is CE6C-C6C5

 Directory of D:\pgrimes\Software\GitHub\bolo_calc_runs\jbolo\ChOpt_Report


 Directory of D:\pgrimes\Software\GitHub\bolo_calc_runs\jbolo\ChOpt_Report\yamls

03/17/2025  14:55    <DIR>          .
03/17/2025  14:55    <DIR>          ..
03/17/2025  14:55             8,898 SAT_2025_AoAalt1.yaml
03/17/2025  13:16             8,940 SAT_2025_PoleAlt1.yaml
03/17/2025  13:16             8,899 SAT_2025_PoleAlt1_273K.yaml
               3 File(s)         26,737 bytes
               2 Dir(s)  648,166,838,272 bytes free


File Not Found


# Compare Baffle and Window Temps at Pole
Two options here:
- CMB-S4 baseline, 273K baffle and window
- BICEP3 estimated radiation temperatures - 240K forebaffle, 260K window

In [29]:
# Dictionary of models to use, which includes yaml file for now, but will have results in it later.
SAT_models = {}

# Thin (1.5mm) window, tandelta = 3e-4
SAT_models['Pole_Alt1']={  
    'yaml':'SAT_2025_AoAalt1.yaml'
}

# Thin (1.5mm) window, tandelta = 3e-4
SAT_models['Pole_Alt1_273K']={  
    'yaml':'SAT_2025_PoleAlt1_273K.yaml'
}

# Thin (1.5mm) window, tandelta = 1.2e-4
SAT_models['Pole_Alt1_B3Temps']={  
    'yaml':'SAT_2025_PoleAlt1.yaml'
}


We're going to do this a very simple way, where we explicitly run each sim and plot it before moving on to the next.


In [30]:
models = list(SAT_models.keys())

uhmwpe_tandelta = 1.2e-4
dyneema_tandelta = 2.4e-4

sims={}
# Run each model
for model in models:
    # Set up
    yamlfile = os.path.join('yamls', SAT_models[model]['yaml'])
    sim = yaml.safe_load(open(yamlfile))
    SAT_models[model]['comment'] = 'NETs are in uKrtsec; powers are in pW'
    SAT_models[model]['name'] = model #sim['version']['name']
    SAT_models[model]['date'] = sim['version']['date']
    SAT_models[model]['site'] = sim['sources']['atmosphere']['site']
    SAT_models[model]['pwv'] = sim['sources']['atmosphere']['pwv']
    SAT_models[model]['elev'] = sim['sources']['atmosphere']['elevation']
    SAT_models[model]['wafers'] = sim['bolo_config']['num_wafers_per_tube']
    SAT_models[model]['optically_good_fraction'] = sim['bolo_config']['optically_good_det_fraction']
    
    # Run
    run_optics(sim)
    run_bolos(sim)
    #print_optics(sim,'MF_1_1')
 
    # Pack outputs into the models dictionary
    SAT_models[model]['channels']=sim['outputs'].keys()
    ii = 0
    for ch in SAT_models[model]['channels']:
        SAT_models[model][ch]={}
        net1det = 1e6*sim['outputs'][ch]['NET_C_total']  # use the version with horn-horn correlations
        ndet = sim['channels'][ch]['num_det_per_wafer']  # detectors per wafer
        net_wafer = net1det/np.sqrt(ndet*sim['bolo_config']['yield'])  #Take yield into account, usually 80%
        net_tube = net_wafer/np.sqrt(SAT_models[model]['wafers']*SAT_models[model]['optically_good_fraction'])  #Take yield into account, usually 80%
        ms = 1/net_tube**2
        SAT_models[model][ch]['P_opt'] = 1e12*sim['outputs'][ch]['P_opt']   #convert to pW from W
        SAT_models[model][ch]['NET_wafer'] = net_wafer   #convert to uKrtsec from Krtsec
        SAT_models[model][ch]['NET_tube'] = net_tube
        SAT_models[model][ch]['mapping_speed'] = ms
        SAT_models[model][ch]['NET_C'] = net1det
        SAT_models[model][ch]['corr_factor'] = sim['outputs'][ch]['corr_factor']
        SAT_models[model][ch]['NET_NC']=1e6*sim['outputs'][ch]['NET_NC_total']
        chnum = sim['channels'][ch]['chnum']
        SAT_models[model][ch]['Lyot_abs'] = sim['optical_elements']['lyot']['absorption'][chnum]
        ii += 1
        
# Print a table for each model
for model in models:
    print(SAT_models[model]['name'], end=', ')
    print(SAT_models[model]['site'], end=', ')
    print('pwv =',SAT_models[model]['pwv'],'u', end=', ')
    print('elev =',SAT_models[model]['elev'])
    #print('            NET_NC   corr_fact NET_C    NET_wafer  NET_tube map_speed  ms_ratio     P_opt')
    print('            NET_NC   ms_ratio     P_opt')
    for ch in SAT_models[model]['channels']:
        msratio = SAT_models[model][ch]['mapping_speed']/SAT_models['Pole_Alt1'][ch]['mapping_speed']
        print(ch.ljust(10),end='')
#        print('{0:8.1f}  {1:8.3f} {2:8.1f}  {3:8.1f} {4:8.2f}  {5:10.2e}  {6:8.2f} {7:10.2f}'.format(SAT_models[model][ch]['NET_NC'], 
#                                                                                           SAT_models[model][ch]['corr_factor'],
#                                                                                           SAT_models[model][ch]['NET_C'],
#                                                                                           SAT_models[model][ch]['NET_wafer'], 
#                                                                                           SAT_models[model][ch]['NET_tube'],
#                                                                                           SAT_models[model][ch]['mapping_speed'],
#                                                                                           msratio,
#                                                                                           SAT_models[model][ch]['P_opt']))
        print('{0:8.1f}  {1:8.2f} {2:10.2f}'.format(SAT_models[model][ch]['NET_NC'], 
                                                    msratio,
                                                    SAT_models[model][ch]['P_opt']))

    print()
    
    

Pole_Alt1, Pole, pwv = 321 u, elev = 50
            NET_NC   ms_ratio     P_opt
LF_1         183.2      1.00       0.43
LF_2         171.8      1.00       1.89
MF_1_1       282.3      1.00       3.00
MF_2_1       243.8      1.00       2.93
MF_1_2       286.3      1.00       4.18
MF_2_2       306.6      1.00       4.45
HF_1         622.0      1.00       9.88
HF_2        1502.8      1.00      12.46

Pole_Alt1_273K, Pole, pwv = 321 u, elev = 50
            NET_NC   ms_ratio     P_opt
LF_1         166.6      1.21       0.38
LF_2         156.4      1.21       1.69
MF_1_1       243.3      1.35       2.52
MF_2_1       200.0      1.49       2.32
MF_1_2       209.4      1.87       2.81
MF_2_2       220.6      1.93       2.92
HF_1         440.7      2.00       6.43
HF_2        1080.4      1.94       8.42

Pole_Alt1_B3Temps, Pole, pwv = 321 u, elev = 50
            NET_NC   ms_ratio     P_opt
LF_1         162.0      1.28       0.37
LF_2         153.6      1.25       1.65
MF_1_1       240.0      1